In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('cleared_dataset.pqt')
df

,timestamp,user_id,product_id,is_sold,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,...,feature_1335,feature_1336,feature_1337,feature_1338,feature_1339,feature_1340,feature_1341,feature_1342,feature_1343,feature_1344
0,2023-01-15,1010508,1,0,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,-0.372165,-0.325964,-1.030499,-0.234190,-0.244455,0.269814
1,2023-01-15,1010508,2,0,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,-0.372165,-0.325964,-1.030499,-0.234190,-0.244455,1.535497
2,2023-01-15,1010508,10,0,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,-0.372165,-0.325964,-1.030499,-0.234190,-0.244455,0.119081
3,2023-01-15,1025935,2,0,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,-0.372165,-0.325964,-0.863126,4.067207,-0.244455,1.535497
4,2023-01-15,1025935,10,0,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,-0.372165,-0.325964,-0.863126,4.067207,-0.244455,0.119081
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
861195,2023-04-30,6215106,0,1,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,NaN,-0.325964,NaN,-0.234190,-0.244455,-0.493016
861196,2023-04-30,6215106,2,1,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,NaN,-0.325964,NaN,-0.234190,-0.244455,1.119107
861197,2023-04-30,6215106,10,1,-0.361854,-0.172997,-0.134458,-0.180080,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,NaN,-0.325964,NaN,-0.234190,-0.244455,0.324334
861198,2023-04-30,6787368,6,1,-0.361854,-0.172997,-0.134458,6.244622,-0.230458,-0.201765,...,-0.330765,-0.155774,-0.123948,-0.188584,NaN,-0.325964,NaN,-0.234190,-0.244455,-0.698873


In [3]:
def target_constructor(df, id_product):
    """
    Разделяет данные для конкретного product_id на Train (прошлые месяцы) и OOT (последний месяц).
    """

    # Фильтруем датасет по нужному продукту
    df_prod = df[df['product_id'] == id_product].copy()
    
    # Проверка: если данных по продукту нет
    if df_prod.empty:
        print(f"Ошибка: Продукт {id_product} не найден в датасете.")
        return None, None, None, None

    # Убеждаемся, что timestamp — это дата
    df_prod['timestamp'] = pd.to_datetime(df_prod['timestamp'])
    
    # Создаем вспомогательную колонку 'period' (год-месяц) для удобного поиска последнего месяца
    df_prod['period'] = df_prod['timestamp'].dt.to_period('M')
    
    # Находим последний месяц (максимальный период)
    last_period = df_prod['period'].max()
    
    # Разделяем на Train (всё кроме последнего) и OOT (только последний)
    mask_oot = df_prod['period'] == last_period
    
    df_train_full = df_prod[~mask_oot] # Все прошлые месяцы
    df_oot_full = df_prod[mask_oot]    # Последний месяц
    
    # Формируем X и y
    # X - удаляем таргет (и вспомогательную колонку period)
    # y - только таргет
    x_train = df_train_full.drop(columns=['is_sold', 'period'])
    y_train = df_train_full['is_sold']
    
    x_oot = df_oot_full.drop(columns=['is_sold', 'period'])
    y_oot = df_oot_full['is_sold']
    
    # --- БЛОК ВЫВОДА ИНФОРМАЦИИ (PRINT) ---
    print("="*60)
    print(f"СТАТИСТИКА ПО ПРОДУКТУ: {id_product}")
    print("="*60)
    
    # Размеры выборок
    print(f"Размер x_train: {x_train.shape[0]} строк")
    print(f"Размер y_train: {y_train.shape[0]} строк")
    print(f"Размер x_oot:   {x_oot.shape[0]} строк")
    print(f"Размер y_oot:   {y_oot.shape[0]} строк")
    print("-" * 30)
    
    # Информация о периодах
    # Берем последний месяц из трейна (если трейн не пустой)
    if not df_train_full.empty:
        max_train_date = df_train_full['timestamp'].max()
        print(f"Последний месяц в TRAIN: {max_train_date.strftime('%B %Y')}")
    else:
        print("TRAIN пуст (возможно, у продукта данные только за 1 месяц)")

    # Месяц OOT
    print(f"Месяц OOT (Test):        {last_period.strftime('%B %Y')}")
    print("-" * 30)
    
    # Статистика по is_sold (конверсия) по месяцам
    print("Распределение is_sold (процент продаж) по месяцам:")
    
    # Объединяем обратно временно для красивого группирования, или используем исходный df_prod
    stats = df_prod.groupby('period')['is_sold'].agg(['count', 'mean'])
    stats['mean'] = (stats['mean'] * 100).round(2)
    stats.columns = ['Кол-во записей', 'Процент продаж %']
    
    # Помечаем, какая строка попала в OOT
    stats['Тип выборки'] = stats.index.map(lambda x: 'OOT' if x == last_period else 'TRAIN')
    
    print(stats)
    print("="*60 + "\n")

    return x_train, y_train, x_oot, y_oot

In [11]:
x_train, y_train, x_oot, y_oot = target_constructor(df, 7)

СТАТИСТИКА ПО ПРОДУКТУ: 7
Размер x_train: 42872 строк
Размер y_train: 42872 строк
Размер x_oot:   10038 строк
Размер y_oot:   10038 строк
------------------------------
Последний месяц в TRAIN: March 2023
Месяц OOT (Test):        April 2023
------------------------------
Распределение is_sold (процент продаж) по месяцам:
         Кол-во записей  Процент продаж % Тип выборки
period                                               
2023-01            6935             20.39       TRAIN
2023-02           15019             18.93       TRAIN
2023-03           20918             19.72       TRAIN
2023-04           10038             32.55         OOT



In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gc
from tqdm import tqdm
from catboost import CatBoostClassifier
from sklearn.metrics import roc_auc_score, average_precision_score

def rfe_with_catboost(X, y, step=1):
    """
    Recursive Feature Elimination.
    1. Обучает модель.
    2. Считает метрики.
    3. Выкидывает 'step' самых слабых фичей.
    4. Повторяет, пока не останется 1 фича.
    
    Returns:
        stats_df: DataFrame с историей (итерация, метрики, удаленная фича).
        remaining_features: список оставшихся фичей (если остановили раньше).
    """
    
    # --- 1. ПОДГОТОВКА ДАННЫХ И СПЛИТ ПО ВРЕМЕНИ ---
    print("⏳ Подготовка данных и разделение по времени (Time Split)...")
    
    # Убеждаемся, что timestamp в формате даты
    X = X.copy()
    X['timestamp'] = pd.to_datetime(X['timestamp'])
    
    # Находим последний месяц
    X['period'] = X['timestamp'].dt.to_period('M')
    last_period = X['period'].max()
    
    # Маска для валидации (последний месяц)
    mask_val = X['period'] == last_period
    
    # Разделяем (технические колонки timestamp и period оставляем для фильтрации, 
    # но потом удалим из обучения)
    X_tr_full = X[~mask_val]
    y_tr = y[~mask_val]
    
    X_val_full = X[mask_val]
    y_val = y[mask_val]
    
    print(f"Train: {X_tr_full.shape[0]} строк | Val (OOT): {X_val_full.shape[0]} строк ({last_period})")
    
    # --- 2. ОПРЕДЕЛЕНИЕ ФИЧЕЙ ---
    # Исключаем технические колонки из обучения
    ignore_cols = ['timestamp', 'period', 'user_id', 'product_id']
    # Также убираем те, которых нет в датасете (на всякий случай)
    ignore_cols = [c for c in ignore_cols if c in X.columns]
    
    current_features = [c for c in X.columns if c not in ignore_cols]
    
    # --- 3. НАСТРОЙКИ CATBOOST (БЫСТРЫЕ) ---
    params = {
        'iterations': 150,          # Чуть больше, чтобы успел понять важность
        'depth': 4,                 
        'learning_rate': 0.1,
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'task_type': 'GPU',         # Включаем GPU
        'devices': '0',
        'verbose': False,
        'allow_writing_files': False,
        'random_seed': 42
    }
    
    history_records = []
    
    # Прогресс-бар (идем от полного набора к 1 фиче)
    # total вычисляем примерно, т.к. шаг может меняться
    pbar = tqdm(total=len(current_features) // step)
    
    iteration = 0
    
    while len(current_features) > 0:
        iteration += 1
        
        # --- A. ОБУЧЕНИЕ ---
        # Выделяем категории
        cats = [c for c in current_features if X[c].dtype == 'object' or X[c].dtype.name == 'category']
        
        model = CatBoostClassifier(**params, cat_features=cats)
        
        model.fit(
            X_tr_full[current_features], y_tr,
            eval_set=(X_val_full[current_features], y_val),
            early_stopping_rounds=20,
            verbose=False
        )
        
        # --- B. МЕТРИКИ ---
        # Предсказываем на OOT (валидации)
        preds_proba = model.predict_proba(X_val_full[current_features])[:, 1]
        
        roc = roc_auc_score(y_val, preds_proba)
        pr = average_precision_score(y_val, preds_proba)
        
        # --- C. ВАЖНОСТЬ ПРИЗНАКОВ ---
        # GetFeatureImportance по умолчанию возвращает "PredictionValuesChange" (для ранжирования ок)
        # Можно использовать type='LossFunctionChange' для большей точности, но это дольше
        fi = model.get_feature_importance()
        
        # Создаем серию: индекс=имя фичи, значение=важность
        fi_series = pd.Series(fi, index=current_features).sort_values(ascending=True)
        
        # --- D. УДАЛЕНИЕ ХУДШИХ ---
        # Берем 'step' самых худших (первых в сортировке)
        worst_features = fi_series.head(step).index.tolist()
        
        # Записываем в лог
        # Если выкидываем пачкой, в поле 'dropped_feature' запишем их через запятую или "batch"
        dropped_name = worst_features[0] if step == 1 else f"{len(worst_features)} features"
        
        record = {
            'iteration': iteration,
            'features_count': len(current_features),
            'roc_auc': roc,
            'pr_auc': pr,
            'dropped_feature': dropped_name,
            'worst_importance': fi_series.values[0] # Важность самого худшего
        }
        history_records.append(record)
        
        # Обновляем список фичей (убираем худшие)
        current_features = [f for f in current_features if f not in worst_features]
        
        # Чистим память
        gc.collect()
        pbar.update(1)
        
        # Если фич не осталось (на последнем шаге), выходим
        if not current_features:
            break

    pbar.close()
    
    # Собираем DataFrame
    stats_df = pd.DataFrame(history_records)
    
    print("✅ RFE завершен.")
    return stats_df

In [ ]:
# step=1 -> Очень долго, но максимально точно (удаляет по 1)
# step=10 -> В 10 раз быстрее, удаляет пачками по 10 штук
# Если у тебя 1400 фичей, начни со step=20, иначе будешь ждать 4 часа.

stats_df = rfe_with_catboost(x_train, y_train, step=1)

# Сохраним табличку
stats_df.to_csv('rfe_metrics_history_7.csv', index=False)
print(stats_df.head())

⏳ Подготовка данных и разделение по времени (Time Split)...
Train: 21954 строк | Val (OOT): 20918 строк (2023-03)


  3%|████                                                                                                                                          | 37/1290 [02:07<1:11:39,  3.43s/it]

In [ ]:
def plot_rfe_results(stats_df):
    plt.figure(figsize=(14, 6))
    
    # Так как итерации идут от "Все фичи" к "Мало фичей", 
    # график может быть интуитивнее, если по оси X будет "Количество оставшихся фичей"
    # Но ты просил "Номер итерации", сделаем так.
    
    x_axis = stats_df['iteration']
    
    # График 1: ROC AUC
    plt.plot(x_axis, stats_df['roc_auc'], label='ROC AUC', color='blue', linewidth=2)
    
    # График 2: PR AUC
    plt.plot(x_axis, stats_df['pr_auc'], label='PR AUC', color='red', linewidth=2)
    
    plt.title('Изменение метрик при удалении признаков (RFE)', fontsize=14)
    plt.xlabel('Номер итерации (удаление худших)', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.7)
    plt.legend(fontsize=12)
    
    # Найдем максимум
    max_roc = stats_df['roc_auc'].max()
    max_roc_iter = stats_df.loc[stats_df['roc_auc'].idxmax(), 'iteration']
    
    plt.axvline(x=max_roc_iter, color='green', linestyle=':', label='Max ROC')
    plt.text(max_roc_iter, max_roc, f' Max: {max_roc:.4f}', color='green', fontweight='bold')
    
    plt.show()

# Рисуем
plot_rfe_results(stats_df)

In [ ]:
best_features_list = stats_df.tail(30)['dropped_feature'].tolist()
best_features_list